<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/13.%20Subword%20Tokenizer/%2013_01_byte_pair_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13-01 바이트 페어 인코딩(Byte Pair Encoding, BPE)

만약 기계가 모르는 단어가 등장하면 그 단어를 단어 집합에 없는 단어란 의미에서 해당 토큰을 UNK(Unknown Token)라고 표현합니다. 

기계가 문제를 풀 때 모르는 단어가 등장하면 (사람도 마찬가지지만) 주어진 문제를 푸는 것이 까다로워집니다. 

이와 같이 모르는 단어로 인해 문제를 푸는 것이 까다로워지는 상황을 OOV(Out-Of-Vocabulary) 문제

##  여기서는 OOV(Out-Of-Vocabulary) 문제를 완화하는 대표적인 서브워드 분리 알고리즘인 BPE(Byte Pair Encoding) 알고리즘을 소개

# 1. BPE(Byte Pair Encoding)

1) ZabdZabac

Z=aa

2) ZYdZYac

Y=ab

Z=aa

3) XdXac

X=ZY

Y=ab

Z=aa

# 2. 자연어 처리에서의 BPE(Byte Pair Encoding)


BPE을 요약하면, 글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용합니다. 우선 훈련 데이터에 있는 단어들을 모든 글자(chracters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합

### 1) 기존의 접근

그리고 이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면 기계는 이 단어를 학습한 적이 없으므로 해당 단어에 대해서 제대로 대응하지 못하는 OOV 문제가 발생


### 2) BPE 알고리즘을 사용한 경우

위의 딕셔너리에 BPE를 적용해봅시다. 우선 딕셔너리의 모든 단어들을 글자(chracter) 단위로 분리

### . 이제부터 딕셔너리는 자신 또한 업데이트되며 앞으로 단어 집합을 업데이트하기 위해 지속적으로 참고되는 참고 자료의 역할

# dictionary

l o w : 5,  

l o w e r : 2, 

n e w e s t : 6, 
 
w i d e s t : 3

딕셔너리를 참고로 한 초기 단어 집합(vocabulary)을 아래와 같습니다. 간단히 말해 초기 구성은 글자 단위로 분리된 상태입니다.

BPE의 특징은 알고리즘의 동작을 몇 회 반복(iteration)할 것인지를 사용자가 정한다는 점입니다. 여기서는 총 10회를 수행한다고 가정합니다. 다시 말해 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 총 10회 반복

1회 빈도수가 9로 가장 높은 (e, s)의 쌍을 es로 통합

2회 - 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합

3회 - 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합

----

### 3) 코드 실습하기







https://wikidocs.net/22592

In [ ]:

import re, collections
from IPython.display import display, Markdown, Latex

BPE을 몇 회 수행할 것인지를 정합니다. 여기서는 10회

In [ ]:

num_merges = 10

In [ ]:

dictionary = {'l o w ' : 5,
         'l o w e r ' : 2,
         'n e w e s t ':6,
         'w i d e s t ':3
         }

. 알고리즘은 위에서 설명했던 것과 동일하게 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정으로 num_merges회 반복

In [ ]:
def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', 'e'): 8, ('e', 'r'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', 'e'): 2, ('e', 'r'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3}
new merge: ('w', 'i')
dictionary: {'low ': 5, 'low e r ': 2, 'newest ': 6, 'wi d est ': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('wi', 'd'): 3, ('d', 'est'): 3}
new merge: ('wi', 'd')
dictionary: {'low ': 5, 'low e r ': 2, 'newest ': 6, 'wid est ': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('wid', 'est'): 3}
new merge: ('wid', 'est')
dictionary: {'low ': 5, 'low e r ': 2, 'newest ': 6, 'widest ': 3}


e와 s의 쌍은 초기 단어 집합에서 총 9회 등장했습니다. 그렇기 때문에 es로 통합됩니다. 그 다음으로는 es와 t의 쌍을, 그 다음으로는 est와 </w>의 쌍을 통합시킵니다. 빈도수가 가장 높은 순서대로 통합하는 이 과정을 총 num_merges회 반복한 것입니다.

In [ ]:

print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('l', 'o'): 2, ('lo', 'w'): 3, ('n', 'e'): 4, ('ne', 'w'): 5, ('new', 'est'): 6, ('w', 'i'): 7, ('wi', 'd'): 8, ('wid', 'est'): 9}


## 4) OOV에 대처하기


In [ ]:

def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('',)
    display(Markdown("__word split into characters:__ {}".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '':
        word = word[:-1]
    elif word[-1].endswith(''):
        word = word[:-1] + (word[-1].replace('',''),)

    return word
     

단어 'loki'가 들어오면 BPE 알고리즘 해당 단어를 어떻게 분리

In [ ]:
encode("loki")


__word split into characters:__ ('l', 'o', 'k', 'i', '')

__Iteration 1:__

bigrams in the word: {('k', 'i'), ('o', 'k'), ('i', ''), ('l', 'o')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '')


__Iteration 2:__

bigrams in the word: {('k', 'i'), ('i', ''), ('lo', 'k')}
candidate for merging: ('k', 'i')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [ ]:
encode("lowest")


__word split into characters:__ ('l', 'o', 'w', 'e', 's', 't', '')

__Iteration 1:__

bigrams in the word: {('s', 't'), ('o', 'w'), ('t', ''), ('w', 'e'), ('l', 'o'), ('e', 's')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '')


__Iteration 2:__

bigrams in the word: {('o', 'w'), ('t', ''), ('l', 'o'), ('w', 'es'), ('es', 't')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '')


...

bigrams in the word: {('lo', 'w'), ('est', ''), ('w', 'est')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est', '')


__Iteration 5:__

bigrams in the word: {('low', 'est'), ('est', '')}
candidate for merging: ('low', 'est')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [ ]:
encode("lowing")


__word split into characters:__ ('l', 'o', 'w', 'i', 'n', 'g', '')

__Iteration 1:__

bigrams in the word: {('i', 'n'), ('n', 'g'), ('o', 'w'), ('w', 'i'), ('l', 'o'), ('g', '')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '')


__Iteration 2:__

bigrams in the word: {('n', 'g'), ('i', 'n'), ('w', 'i'), ('lo', 'w'), ('g', '')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '')


__Iteration 3:__

bigrams in the word: {('low', 'i'), ('g', ''), ('i', 'n'), ('n', 'g')}
candidate for merging: ('low', 'i')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [ ]:
encode("highing")


__word split into characters:__ ('h', 'i', 'g', 'h', 'i', 'n', 'g', '')

__Iteration 1:__

bigrams in the word: {('i', 'n'), ('n', 'g'), ('g', 'h'), ('h', 'i'), ('g', ''), ('i', 'g')}
candidate for merging: ('i', 'n')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'g')